### Demostration for Researcher Use Case
This notebook contain a prospective analysis of bird sighting variation due changing air quaility in Oregon. Our analysis focuses on the 2020 Oregon wildfires. Understanding how bird observations change with worsening air quaility could offer insight into how bird populations respond to wildfires. Our main goal is data exploration. We hope that any interesting findings could be used by researchers in future studies. Secondly, we hope to demonstrate some of the functionality of Altair, a descriptive statiscal visualization library, within the context of ecological research.

#### Import Data and Initial Packages

In [ ]:
# First let's import the necessary packages for the inital data processing
import numpy as np
import pandas as pd

# We will also import our two python modules, which contain functions for adding estimate AQI to the sightings
import air_quality_knn
import data_cleaning


In [ ]:
# now we will read in the Ebird and air quality data as pandas dataframes (which is necessary for Altair)

bird_df = pd.read_csv("https://bernease.s3-us-west-2.amazonaws.com/hold/cse583_au20_ebird/ebird_residents_OR_2020.csv")
air_df = pd.read_csv("https://raw.githubusercontent.com/emilysellinger/Phoenix/main/phoenix/data/Daily_Avg_PM2.5_Location.csv")

In [ ]:
# Now that we have the data as pandas dataframes, we can pass them to our functions in our python modules
# The first function will use k nearest neighbors regression to estimate the PM 2.5 level at the latitute
# and longitude of the bird sighting based on the levels at nearby air stations

bird_df = air_quality_knn.air_quality_knn(air_df, bird_df)

# The second function will use the estimated PM 2.5 level to determine the Air Quality Index (AQI). The EPA has
# six AQI categories, more information can be found in our function documentation as well as on the EPA website

bird_df = data_cleaning.assign_aqicat(bird_df)

In [ ]:
# We can check that the new columns have been added by looking at the dataframe
bird_df.head()

In [ ]:
# We can also get a summary of the columns using the pandas function describe
bird_df.describe()

In [ ]:
# Because the ranges on both the PM 2.5 levels and the observation counts is large, we will log transform both of
# those columns

bird_df['log_avg_PM'] = np.log(bird_df['Avg_PM2.5'])

bird_df['log_obs_count'] = np.log(bird_df['observation count'])


In [ ]:
# Because the data set is so large, we will focuse our visualizations on immediately before and after the fires
# To do this, we will create a smaller dataset that only includes dates after July 31, 2020
red_bird_df = bird_df.loc[bird_df['observation date'] > '2020-07-31']
red_bird_df.head()

#### Inital Visualizations


In [ ]:
# First let's import the necessary packages for data exploration/visualization
import altair as alt

In [ ]:
# Note: If you choose not to call in the data via url, you may experience lag when creating Altair graphs. Also, 
# if the data frame is larger than 5000 rows, you will need to disable the max rows setting in Altair

alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(red_bird_df).mark_line().encode(
    x='observation date',
    y='log_obs_count',
    color='order'
)
# It's hard to see any trends in a format like this so let's graph the data a different way 

In [ ]:
# Let's look at the log of PM2.5 level versus log of observation count for each county by bird taxonomic order
alt.Chart(red_bird_df).mark_point().encode(
    column='county',
    x='log_avg_PM',
    y='log_obs_count',
    color='order'
)
# even after transforming the data, there is still some issues with clumping that is likely related to how data
# is collected in Ebird (not all users include the number of individual birds they see, in cases where that
# is the case, Ebird just records an X for presence, we substituted that X for 1. As such, there are a lot of
# observation counts equal to 1, which might not be reflective of the true number of individuals sighted)

In [ ]:
# Let's look at the log of observation counts versus AQI index by county
alt.Chart(red_bird_df).mark_boxplot().encode(
    column='county',
    x=alt.X('AQI_Category', sort=['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy',
                               'Very Unhealthy', 'Hazardous']),
    y='log_obs_count'
)

# There seems to be some variation in observation counts across counties, which makes sense as the fires
# were not evenly distributed across the state

In [ ]:
# Let's look at the log of observation counts versus AQI index by bird taxonomic order
alt.Chart(red_bird_df).mark_boxplot().encode(
    column='order',
    x=alt.X('AQI_Category', sort=['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy',
                               'Very Unhealthy', 'Hazardous']),
    y='log_obs_count'
)
# It seems that using AQI categories may be more meaningful of an analysis than the raw PM2.5 level due to
# error at a more fine scale measurement. However, there are still a lot of outliers, especially for the 
# "Good" AQI days

In [ ]:
# Looking at the box plots, there seems to variation between number of observations, AQI, and taxonomic order

#### Statistical Models
Based on our initial data exploration, we believe daily observation counts vary with air quaility with county and taxonomic order also affecting sighthings. Given the noise in the PM2.5 levels, using AQI as a categorical variable might be more appropriate.

In [ ]:
# first let's import the scipy statistics module, which will allow us to 
import scipy.stats as stats

In [ ]:
# Now we will break our bird_df into observations by AQI category "Good", "Moderate", 
# "Unhealthy for Sensative Groups", "Unhealthy", "Very Unhealthy", "Hazardous"
# Like R, python allows for easy subsection of dataframes

good_counts = red_bird_df[(red_bird_df['AQI_Category'] == 'Good')]

mod_counts = red_bird_df[(red_bird_df['AQI_Category'] == 'Moderate')]

usg_counts = red_bird_df[(red_bird_df['AQI_Category'] == 'Unhealthy for Sensitive Groups')]

unhealthy_counts = red_bird_df[(red_bird_df['AQI_Category'] == 'Unhealthy')]

vu_counts = red_bird_df[(red_bird_df['AQI_Category'] == 'Very Unhealthy')]

haz_counts = red_bird_df[(red_bird_df['AQI_Category'] == 'Hazardous')]

In [ ]:
# With the subset data, we can perform a one-way ANOVA
F, p = stats.f_oneway(good_counts['log_obs_count'],mod_counts['log_obs_count'], usg_counts['log_obs_count'], 
                      unhealthy_counts['log_obs_count'], vu_counts['log_obs_count'], haz_counts['log_obs_count'])
print('F-Statistic=%.3f, p=%.3f' % (F, p))

#### Summary and Next Steps
A simple one-way ANOVA found that the average log of daily bird observations was significantly different among AQI categories. Following a significant p-value, further ad hoc tests should be completed to determine which of the averages are significantly different from one another. Also, researchers could dive further into the relationship between county, order, and PM2.5 levels. To further untangle the relationship between order, the inclusion of primary habitat type might be useful. Even within the same order, habitat preferences may differ and certain habitats, such as forest edges.